In [ ]:
from seleniumwire import webdriver
import time

url = r'https://f1.tfeed.net/live'

# Create a new instance of the Chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
driver = webdriver.Chrome( options=options)

# Go to the URL
driver.get(url)

# Wait for the page to load completely
# Add your specific wait conditions here if needed

# Define a set to store already processed URLs
processed_urls = set()

while True:
    # Access requests via the `requests` attribute
    for request in driver.requests:
        if request.response and 'tt.js' in request.url and request.url not in processed_urls:
            print(request.url)
            processed_urls.add(request.url)


    # Wait for a short interval before checking again
    time.sleep(0.002)